# Thuật toán XGBoost for regression

**1. Khởi tạo mô hình**
- Giá trị dự đoán ban đầu thường được khởi tạo bằng giá trị trung bình của dữ liệu đầu ra.

**2. Cơ sở toán học**
- Hàm loss: $L(y, f) = \sum_{i = 1}^N (y_i - f(x_i))^2$
- Tuy nhiên để hạn chế overfitting, XGBoost thêm thành phần regularization: $L(y, f) = \sum_{i = 1}^N (y_i - f(x_i))^2 + \Omega (f)$
    + $\Omega(f) = \gamma T + \frac{1}{2} \lambda \sum_{j = 1}^T w_j$.    $ \ \ \ \ \ T$ là số nút lá

- Sử dụng phép xấp xỉ Taylor đến bậc 2 thu được:
    $$ L =  \sum_{i=1}^{n} \left[ g_i f(\mathbf{x}_i) + \frac{1}{2} h_i f^2(\mathbf{x}_i) \right] + \Omega(f)$$

    + $$g_i = \frac{\partial L(y_i, f(x_i))}{\partial f(x_i)} $$
    + $$h_i = \frac{\partial^2 L(y_i, f(x_i))}{\partial^2 f(x_i)} $$

**3.Xây dựng M mô hình**
- Xây dựng các cây quyết định dựa trên lỗi hiện tại
    + Để phân chia các thuộc tính, ta sử dụng Gain. Trong XGBoost, gain được tính dựa trên đạo hàm bậc 1 (gradient), và bậc 2 (Hessian) của hàm loss 
    + Đạo hàm bậc 1: $$g_i = \frac{\partial L(y_i, f(x_i))}{\partial f(x_i)} $$
    + Đạo hàm bậc 2: $$h_i = \frac{\partial^2 L(y_i, f(x_i))}{\partial^2 f(x_i)} $$
    + Tính gain: 
        $$
        \text{Gain} = \frac{1}{2} \left( \frac{(G_l)^2}{H_l + \lambda} + \frac{(G_r)^2}{H_r + \lambda} - \frac{(G)^2}{H + \lambda} \right) - \gamma
        $$

        - $G_l$ và $G_r$ là tổng gradient cho phần bên trái và phải của phân chia.
        - $H_l$ và $H_r$ là tổng hessian cho phần bên trái và phải.
        - $G$ và $H$ là tổng gradient và hessian trước khi chia.
        - $\lambda$ là tham số điều chỉnh regularization (tránh overfiting).
        - $\gamma$ là tham số điều chỉnh việc phân chia, có thể hiểu là nếu Gain > $\gamma$ thì tiếp tục phân chia.
    
    + Cập nhật mô hình hiện tại với cây mới:

        $$
        f_{m+1}(x) = f_m(x) + \eta \cdot h_m(x)
        $$

        Trong đó:
        - $\eta$ là learning rate, điều chỉnh sự ảnh hưởng của môi hình mới đến mô hình hiện tại
        - $h_m(x)$ là dự đoán của cây quyết định mới xây dựng dựa trên lỗi ở bên trên



### Tài liệu tham khảo:
 https://arxiv.org/pdf/1603.02754

# Ví dụ python

In [7]:
import numpy as np
import pandas as pd
import math


class TreeBooster:
    def __init__(self, X, gradients, hessians, max_depth, min_child_weight, reg_lambda, gamma, idxs=None):
        self.max_depth = max_depth
        self.min_child_weight = min_child_weight
        self.reg_lambda = reg_lambda
        self.gamma = gamma
        
        if idxs is None: idxs = np.arange(len(gradients))
        self.X, self.gradients, self.hessians, self.idxs = X, gradients, hessians, idxs
        self.n, self.c = len(idxs), X.shape[1]
        self.value_predict = -self.gradients[self.idxs].sum() / (self.hessians[self.idxs].sum() + self.reg_lambda)
        self.best_score_so_far = 0.
        if self.max_depth > 0:
            self._maybe_insert_child_nodes()

    def _maybe_insert_child_nodes(self):
        for i in range(self.c):
            self._find_better_split(i)
        if self.is_leaf():
            return
        x = self.X[self.idxs, self.split_feature_idx]
        left_idx = self.idxs[x <= self.threshold]
        right_idx = self.idxs[x > self.threshold]
        self.left = TreeBooster(self.X, self.gradients, self.hessians, self.max_depth - 1, self.min_child_weight, self.reg_lambda, self.gamma, left_idx)
        self.right = TreeBooster(self.X, self.gradients, self.hessians, self.max_depth - 1, self.min_child_weight, self.reg_lambda, self.gamma, right_idx)

    def is_leaf(self):
        return self.best_score_so_far == 0.

    def _find_better_split(self, feature_idx):
        x = self.X[self.idxs, feature_idx]
        g, h = self.gradients[self.idxs], self.hessians[self.idxs]
        sort_idx = np.argsort(x)
        sort_g, sort_h, sort_x = g[sort_idx], h[sort_idx], x[sort_idx]
        sum_g, sum_h = g.sum(), h.sum()
        sum_g_right, sum_h_right = sum_g, sum_h
        sum_g_left, sum_h_left = 0., 0.

        for i in range(0, self.n - 1):
            g_i, h_i, x_i, x_i_next = sort_g[i], sort_h[i], sort_x[i], sort_x[i + 1]
            sum_g_left += g_i
            sum_h_left += h_i
            sum_g_right -= g_i
            sum_h_right -= h_i
            if sum_h_left < self.min_child_weight or x_i == x_i_next:
                continue
            if sum_h_right < self.min_child_weight:
                break

            gain = 0.5 * ((sum_g_left**2 / (sum_h_left + self.reg_lambda))
                          + (sum_g_right**2 / (sum_h_right + self.reg_lambda))
                          - (sum_g**2 / (sum_h + self.reg_lambda))) - self.gamma

            if gain > self.best_score_so_far:
                self.split_feature_idx = feature_idx
                self.best_score_so_far = gain
                self.threshold = (x_i + x_i_next) / 2

    def predict(self, X):
        return np.array([self._predict_row(row) for row in X])

    def _predict_row(self, row):
        if self.is_leaf():
            return self.value_predict
        
        child = self.left if row[self.split_feature_idx] <= self.threshold else self.right
        return child._predict_row(row)


class XGBoostRegressor:
    def __init__(self, learning_rate=0.1, max_depth=3, min_child_weight=1, gamma=0, reg_lambda=1, subsample=1.0, num_boost_round=100, random_seed=None):
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_child_weight = min_child_weight
        self.gamma = gamma
        self.reg_lambda = reg_lambda
        self.subsample = subsample
        self.num_boost_round = num_boost_round
        self.rng = np.random.default_rng(seed=random_seed)

    def fit(self, X, y):
        self.base_score = np.mean(y)
        if isinstance(X, pd.DataFrame): X = X.values
        if isinstance(y, pd.Series): y = y.values
        current_predictions = self.base_score * np.ones_like(y)
        self.boosters = []
        for i in range(self.num_boost_round):
            gradients = 2 * (current_predictions - y)
            hessians = 2 * np.ones_like(y)

            sample_idxs = None
            
            # Kỹ thuật subsampling
            if self.subsample == 1.0:
                sample_idxs = None
            else:
                sample_idxs = self.rng.choice(len(y),
                                     size=math.floor(self.subsample * len(y)),
                                     replace=False)
                
            booster = TreeBooster(X, gradients, hessians, self.max_depth, self.min_child_weight, self.reg_lambda, self.gamma, sample_idxs)
            current_predictions += self.learning_rate * booster.predict(X)
            self.boosters.append(booster)

    def predict(self, X):
        if isinstance(X, pd.DataFrame): X = X.values
        return self.base_score + np.sum([self.learning_rate * booster.predict(X) for booster in self.boosters], axis=0)

In [8]:
import pandas as pd
data = pd.read_csv('advertising.csv')
data

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,12.0
3,151.5,41.3,58.5,16.5
4,180.8,10.8,58.4,17.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,14.0
197,177.0,9.3,6.4,14.8
198,283.6,42.0,66.2,25.5


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   Radio      200 non-null    float64
 2   Newspaper  200 non-null    float64
 3   Sales      200 non-null    float64
dtypes: float64(4)
memory usage: 6.4 KB


In [12]:

X = data.drop('Sales', axis=1)
y = data['Sales']

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

xgboost_regressor = XGBoostRegressor(learning_rate=0.1, max_depth=3)
xgboost_regressor.fit(X_train, y_train)

y_pred = xgboost_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'r2 score: ', r2_score(y_pred, y_test))

Mean Squared Error: 1.3847068126542923
r2 score:  0.9378703321189994


In [11]:
import xgboost as xgb
model = xgb.XGBRegressor(
    n_estimators=100,    
    learning_rate=0.1,  
    max_depth=3  
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'r2 score: ', r2_score(y_pred, y_test))

Mean Squared Error: 1.5058464432638747
r2 score:  0.9183606799021481


In [2]:
data = pd.read_csv('car_evaluation.csv')

X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [3]:
X = pd.get_dummies(X)